## Imports

In [ ]:
%load_ext lab_black

import h5py
import os
import numpy as np
from typing import Dict, List, Optional, Tuple

from dataclasses import dataclass
from tqdm.auto import tqdm
from scipy.signal import savgol_filter
from scipy.interpolate import interp2d
from functools import lru_cache


import plotly.graph_objects as go
import plotly.colors as pc
import matplotlib.pyplot as plt


import sys

sys.path.append(r"C:\Users\atully\Code\GitHub\ARPES Code\arpes-code-python")
from arpes_functions import (
    fitting_functions,
    analysis_functions,
    plotting_functions,
    HDF5_loader,
    misc_functions,
    filter_functions,
    tr_functions,
    loading_functions,
    kw_data_loader,
    cnn,
    polygons,
)

colors = pc.qualitative.D3
angstrom = "\u212B"
Theta = "\u0398"
phi = "\u03C6"

In [ ]:
def average_timescans(files, ddir, new_filename):
    datas = []
    for i in range(0, len(files)):
        ad = ARPES_DATA[files[i]]
        datas.append(ad.data)
    data_avg = np.mean(datas, axis=0)
    print(data_avg.shape)

    new_data = data_avg

    new_fn = os.path.join(ddir, new_filename)

    with h5py.File(
        new_fn, "w"
    ) as f:  # Note: 'w' creates a new empty file (or overwrites), use 'r+' to modify an existing file
        f["data"] = new_data.T
        axes_names = [
            "angles",
            "energies",
        ]  # Change these to match your axes labels
        axes = [ad.theta, ad.energy]
        for axis, name in zip(axes, axes_names):
            f[name] = np.atleast_2d(axis).T
        entry_group = f.require_group("entry1")
        entry_group["ScanValues"] = np.atleast_2d(ad.phi_or_time).T
    return new_fn


def sum_timescans(files, ddir, new_filename):
    datas = []
    for i in range(0, len(files)):
        ad = ARPES_DATA[files[i]]
        datas.append(ad.data)
    data_avg = np.sum(datas, axis=0)
    print(data_avg.shape)

    new_data = data_avg

    new_fn = os.path.join(ddir, new_filename)

    with h5py.File(
        new_fn, "w"
    ) as f:  # Note: 'w' creates a new empty file (or overwrites), use 'r+' to modify an existing file
        f["data"] = new_data.T
        axes_names = [
            "angles",
            "energies",
        ]  # Change these to match your axes labels
        axes = [ad.theta, ad.energy]
        for axis, name in zip(axes, axes_names):
            f[name] = np.atleast_2d(axis).T
        entry_group = f.require_group("entry1")
        entry_group["ScanValues"] = np.atleast_2d(ad.phi_or_time).T
    return new_fn

# Load Data

In [ ]:
ddir = r"E:\atully\arpes_data\2023_June\C60\ARPES\FS"
files = []

## CT1: G point ##

## FS0  --> CT1, zero delay
# files = [f"FS0\FS_000_{i}.h5" for i in range(1, 17)]

## FS3  --> CT1, zero delay, top phi range
# files = [f"FS3\FS_003_{i}.h5" for i in range(1, 17)]

## FS2  --> CT1, zero delay, extended phi range
# files = [f"FS2\FS_002_{i}.h5" for i in range(1, 16)]


## CT1: K point ##

## FS1  --> CT1, zero delay, K point in center
# files = [f"FS1\FS_001_{i}.h5" for i in range(1, 6)]

## FS4  --> CT1, zero delay, K point top phi range
# files = [f"FS4\FS_004_{i}.h5" for i in range(1, 6)]

## FS5,6,7  --> CT1, zero delay, K point extended phi range
# files = [f"FS5\FS_005_{i}.h5" for i in range(1, 4)]
# files.extend(f"FS6\FS_006_{i}.h5" for i in range(1, 18))
# files.extend(f"FS7\FS_007_{i}.h5" for i in range(1, 7))

## CT2 ##

## FS8,9,10  --> CT2, right side of BZ
# FS8
# files = [f"FS8\FS_008_{i}.h5" for i in range(1, 15)]
# files = [f"FS8&9&10_Ali_avg.h5" for i in range(1, 2)]
# files.extend(f"FS8\FS_008_{i}.h5" for i in range(15, 24))

# FS9
# files = [f"FS8&9&10_Ali_avg.h5" for i in range(1, 2)]
# files.extend(f"FS9\FS_009_{i}.h5" for i in range(1, 6))

# FS10
# files = [f"FS8&9&10_Ali_avg.h5" for i in range(1, 2)]
# files.extend(f"FS10\FS_010_{i}.h5" for i in range(1, 12))
# files.extend(f"FS10\FS_010_{i}.h5" for i in range(12, 24))

## FS11, 12  --> CT2, left side of BZ
# files = [f"FS11\FS_011_{i}.h5" for i in range(1, 4)]
# files.extend(f"FS12\FS_012_{i}.h5" for i in range(1, 4))


## For Stitching Stats (Left to match Right) ##
# files = [f"FS10\FS_010_{i}.h5" for i in range(1, 7)]  # right
# files = [f"FS11\FS_011_{i}.h5" for i in range(1, 4)]  # left
# files.extend(f"FS12\FS_012_{i}.h5" for i in range(1, 4))


ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=kx, phi_or_time=ky, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

In [ ]:
# average_timescans(files, ddir, "FS0\FS0_Ali_avg.h5")
# average_timescans(files, ddir, "FS1\FS1_Ali_avg.h5")
# average_timescans(files, ddir, "FS2\FS2_Ali_avg.h5")
# average_timescans(files, ddir, "FS3\FS3_Ali_avg.h5")
# average_timescans(files, ddir, "FS4\FS4_Ali_avg.h5")
# average_timescans(files, ddir, "FS5\FS5_Ali_avg.h5")

# average_timescans(files, ddir, "FS7\FS5&6&7_Ali_avg.h5")


## CT2 ##
# sum_timescans(files, ddir, "FS8&9&10_Ali_avg.h5")  # right sidee
# sum_timescans(files, ddir, "FS10_Ali_avg.h5")  # 6 cycles right side, for matching stats
# sum_timescans(files, ddir, "FS11&12_Ali_avg.h5")  # left side (6 cycles)

In [ ]:
## Load averaged data ##

ddir = r"E:\atully\arpes_data\2023_June\C60\ARPES\FS"
files = []

## CT1 ##

## G point ##
# files = ["FS0\FS0_Ali_avg.h5"]  # left side: bottom
# files = ["FS3\FS3_Ali_avg.h5"]  # left side: top
# files = ["FS2\FS2_Ali_avg.h5"]  # left side: full

## K point ##
# files = ["FS1\FS1_Ali_avg.h5"]  # right side: bottom
# files = ["FS4\FS4_Ali_avg.h5"]  # right side: top
# files = ["FS7\FS5&6&7_Ali_avg.h5"]  # right side: full

## CT2 ##
# files = ["FS8&9&10_Ali_avg.h5"]
files = ["FS11&12_Ali_avg.h5"]


## Feb Comparison ##
# ddir = r"E:\atully\arpes_data\2023_February\6eV\FS"
# files = []
# files = ["FS1_FS3_Ali_avg.h5"]  # 2.0 eV center energy
# files = ["FS_001_1.h5"]  # 2.0 eV center energy

# This works, but makes dataclass with theta and phi_or_time instead of kx and ky
ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=kx, phi_or_time=ky, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad = ARPES_DATA[files[0]]

In [ ]:
# ad = ARPES_DATA[files[0]]
# for k in ["energy", "theta", "phi_or_time"]:
#     print(f"{k}.shape = {getattr(ad, k).shape}")
# print(f"Data.shape = {ad.data.shape}")

# Analysis

In [ ]:
slice_dim = "y"
# slice_val = 0
int_range = 0.05
xlim = (-18, 16)
ylim = None
x_bin = 2
y_bin = 2

ad.theta = ad.theta - 10.5
xlim = (-18 - 10.5, 16 - 10.5)

# Load Data

for i, v in enumerate([2.95, 2.85, 2.75, 2.65, 2.55, 2.45, 2.35]):  # CT2
    # for i, v in enumerate([2.4, 2.3, 2.2, 2.1, 2.0]):  # CT1
    x, y, d = tr_functions.slice_datacube(
        ad_dataclass=ad,
        slice_dim=slice_dim,
        slice_val=v,
        int_range=int_range,
        xlim=xlim,
        ylim=ylim,
        x_bin=x_bin,
        y_bin=y_bin,
        norm_data=True,
        plot_data=False,
    )

    ## Plot Data Plotly
    fig = tr_functions.thesis_fig(
        title=f"E<sub>K</sub> = {v}",
        xaxis_title=f"{Theta}",
        yaxis_title=f"{phi}",
        equiv_axes=False,
        height=500,
        width=500,
    )

    fig.add_trace(
        go.Heatmap(x=x, y=y, z=analysis_functions.norm_data(d), coloraxis="coloraxis")
    )

    # if xlim is not None:
    #     fig.update_xaxes(range=[xlim[0], xlim[1]], constrain="domain")

    fig.update_coloraxes(cmin=0, cmax=1)

    fig.show()

# Stitch Datasets

In [ ]:
## CT2 -- highest stats

files = ["FS10_Ali_avg.h5"]
# files = ["FS11&12_Ali_avg.h5"]

# This works, but makes dataclass with theta and phi_or_time instead of kx and ky
ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=kx, phi_or_time=ky, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

ad_right = ARPES_DATA[files[0]]
# ad_left = ARPES_DATA[files[0]]

In [ ]:
ad1 = ad_right
ad2 = ad_left

slice_dim = "y"
# slice_val = 0
int_range = 0.05
xlim = (-18, 16)
# xlim = None
ylim = None
x_bin = 2
y_bin = 2

# for i, v in enumerate([2.4, 2.3, 2.2, 2.1, 2.0]):  # CT1
for i, v in enumerate([2.95, 2.85, 2.75, 2.65, 2.55, 2.45, 2.35]):  # CT2
    x1, y1, d1 = tr_functions.slice_datacube(
        ad_dataclass=ad1,
        slice_dim=slice_dim,
        slice_val=v,
        int_range=int_range,
        xlim=xlim,
        ylim=ylim,
        x_bin=x_bin,
        y_bin=y_bin,
        norm_data=True,
        plot_data=False,
    )

    x1 = x1 + 10.5  # for right side

    x2, y2, d2 = tr_functions.slice_datacube(
        ad_dataclass=ad2,
        slice_dim=slice_dim,
        slice_val=v,
        int_range=int_range,
        xlim=xlim,
        ylim=ylim,
        x_bin=x_bin,
        y_bin=y_bin,
        norm_data=True,
        plot_data=False,
    )
    x2 = x2 - 10.5  # for left side

    xs, ys, ds = tr_functions.stitch_and_avg(
        x1,
        y1,
        d1,
        x2,
        y2,
        d2,
        no_avg=True,
    )

    ## Plot Data

    fig = tr_functions.thesis_fig(
        title=f"E<sub>K</sub> = {v}",
        xaxis_title=f"{Theta}",
        yaxis_title=f"{phi}",
        equiv_axes=False,
        height=500,
        width=500,
    )

    # fig.add_trace(
    #     go.Heatmap(
    #         x=x2,
    #         y=y2,
    #         z=analysis_functions.norm_data(d2) * 2,
    #         coloraxis="coloraxis",
    #     )
    # )
    # fig.add_trace(
    #     go.Heatmap(
    #         x=x1,
    #         y=y1,
    #         z=analysis_functions.norm_data(d1),
    #         coloraxis="coloraxis",
    #     )
    # )

    fig.add_trace(go.Heatmap(x=xs, y=ys, z=ds, coloraxis="coloraxis"))

    # if xlim is not None:
    #     fig.update_xaxes(range=[xlim[0], xlim[1]], constrain="domain")

    # fig.update_coloraxes(cmin=0.0, cmax=0.6)

    fig.show()

# Stitch Full CT1 

In [ ]:
slice_val = 2.0
# slice_val = 2.1
# slice_val = 2.2
# slice_val = 2.3
# slice_val = 2.4


slice_dim = "y"
# slice_val = 0
int_range = 0.05
xlim = (-18, 16)
# xlim = None
ylim = None
x_bin = 2
y_bin = 2

## Left Side

In [ ]:
## CT1 -- largest area matching stats

# files = ["FS0\FS0_Ali_avg.h5"]  # bottom
# files = ["FS3\FS3_Ali_avg.h5"]  # top
# files = ["FS2\FS2_Ali_avg.h5"]  # full range

## Match Stats ##
# files = ["FS0\FS0_Ali_avg_forstitch.h5"]  # bottom
# files = ["FS3\FS3_Ali_avg_forstitch.h5"]  # top

# This works, but makes dataclass with theta and phi_or_time instead of kx and ky
ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=kx, phi_or_time=ky, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

# ad_bottom_left = ARPES_DATA[files[0]]
# ad_top_left = ARPES_DATA[files[0]]
# ad_full_left = ARPES_DATA[files[0]]

In [ ]:
## Get Slices ##

x_bottom_left, y_bottom_left, d_bottom_left = tr_functions.slice_datacube(
    ad_dataclass=ad_bottom_left,
    slice_dim=slice_dim,
    slice_val=slice_val,
    int_range=int_range,
    xlim=xlim,
    ylim=ylim,
    x_bin=x_bin,
    y_bin=y_bin,
    norm_data=True,
    plot_data=False,
)

x_top_left, y_top_left, d_top_left = tr_functions.slice_datacube(
    ad_dataclass=ad_top_left,
    slice_dim=slice_dim,
    slice_val=slice_val,
    int_range=int_range,
    xlim=xlim,
    ylim=ylim,
    x_bin=x_bin,
    y_bin=y_bin,
    norm_data=True,
    plot_data=False,
)

x_full_left, y_full_left, d_full_left = tr_functions.slice_datacube(
    ad_dataclass=ad_full_left,
    slice_dim=slice_dim,
    slice_val=slice_val,
    int_range=int_range,
    xlim=xlim,
    ylim=ylim,
    x_bin=x_bin,
    y_bin=y_bin,
    norm_data=True,
    plot_data=False,
)

In [ ]:
## Stitch left side: bottom (FS0), top (FS3) ##

x1, y1, dataslice1 = x_bottom_left, y_bottom_left, d_bottom_left
x2, y2, dataslice2 = (
    x_top_left,
    y_top_left,
    # d_top_left / 2,
    d_top_left / 1.8,
)  # normalize background to each other

xs, ys, ds = tr_functions.stitch_and_avg(
    x1,
    y1,
    dataslice1,
    x2,
    y2,
    dataslice2,
    no_avg=True,
)

## Average with full left side: (FS2) ##

x1, y1, dataslice1 = xs, ys, ds
x2, y2, dataslice2 = x_full_left, y_full_left, d_full_left

xs_2, ys_2, ds_2 = tr_functions.stitch_and_avg(
    x1,
    y1,
    dataslice1,
    x2,
    y2,
    dataslice2,
    no_avg=False,
)

In [ ]:
x_left, y_left, d_left = xs_2 + 10, ys_2, ds_2

In [ ]:
## Plot Data ##
# x_plot, y_plot, d_plot = xs + 10, ys, ds
x_plot, y_plot, d_plot = x_left, y_left, d_left

fig = tr_functions.thesis_fig(
    title=f"E<sub>K</sub> = {slice_val}",
    xaxis_title=f"{Theta}",
    yaxis_title=f"{phi}",
    equiv_axes=False,
    height=500,
    width=500,
)

fig.add_trace(
    go.Heatmap(
        x=x_plot,
        y=y_plot,
        z=analysis_functions.norm_data(d_plot),
        coloraxis="coloraxis",
    )
)

fig.update_coloraxes(cmin=0, cmax=1)

fig.show()

## Right Side

In [ ]:
# CT1 -- largest area matching stats

# files = ["FS1\FS1_Ali_avg.h5"]  # bottom
# files = ["FS4\FS4_Ali_avg.h5"]  # top
# files = ["FS6\FS5&6_Ali_avg.h5"]  # full range


# This works, but makes dataclass with theta and phi_or_time instead of kx and ky
ARPES_DATA: Dict[str, tr_functions.ArpesData] = {}
ARPES_ATTRS: Dict[str, tr_functions.ArpesAttrs] = {}
for file in tqdm(files):
    data, kx, ky, energy = loading_functions.load_hdf5(ddir, file)
    ARPES_DATA[file] = tr_functions.ArpesData(
        data=data, theta=kx, phi_or_time=ky, energy=energy
    )
    ARPES_ATTRS[file] = tr_functions.load_attrs_hdf5(ddir, file)

# ad_bottom_right = ARPES_DATA[files[0]]
# ad_top_right = ARPES_DATA[files[0]]
# ad_full_right = ARPES_DATA[files[0]]

In [ ]:
## Get Slices ##

x_bottom_right, y_bottom_right, d_bottom_right = tr_functions.slice_datacube(
    ad_dataclass=ad_bottom_right,
    slice_dim=slice_dim,
    slice_val=slice_val,
    int_range=int_range,
    xlim=xlim,
    ylim=ylim,
    x_bin=x_bin,
    y_bin=y_bin,
    norm_data=True,
    plot_data=False,
)

x_top_right, y_top_right, d_top_right = tr_functions.slice_datacube(
    ad_dataclass=ad_top_right,
    slice_dim=slice_dim,
    slice_val=slice_val,
    int_range=int_range,
    xlim=xlim,
    ylim=ylim,
    x_bin=x_bin,
    y_bin=y_bin,
    norm_data=True,
    plot_data=False,
)

x_full_right, y_full_right, d_full_right = tr_functions.slice_datacube(
    ad_dataclass=ad_full_right,
    slice_dim=slice_dim,
    slice_val=slice_val,
    int_range=int_range,
    xlim=xlim,
    ylim=ylim,
    x_bin=x_bin,
    y_bin=y_bin,
    norm_data=True,
    plot_data=False,
)

In [ ]:
## Stitch right side: bottom (FS1) & top (FS4) ##

x1, y1, dataslice1 = x_bottom_right, y_bottom_right, d_bottom_right
x2, y2, dataslice2 = x_top_right, y_top_right, d_top_right

xs, ys, ds = tr_functions.stitch_and_avg(
    x1,
    y1,
    dataslice1,
    x2,
    y2,
    dataslice2,
    no_avg=True,
)

## Average with full right side: (FS5) ##

x1, y1, dataslice1 = xs, ys, ds
x2, y2, dataslice2 = x_full_right, y_full_right, d_full_right

xs_2, ys_2, ds_2 = tr_functions.stitch_and_avg(
    x1,
    y1,
    dataslice1,
    x2,
    y2,
    dataslice2,
    no_avg=False,
)

In [ ]:
x_right, y_right, d_right = xs_2 + 25, ys_2, ds_2

In [ ]:
## Plot Data ##
x_plot, y_plot, d_plot = x_right, y_right, d_right

fig = tr_functions.thesis_fig(
    title=f"E<sub>K</sub> = {slice_val}",
    xaxis_title=f"{Theta}",
    yaxis_title=f"{phi}",
    equiv_axes=False,
    height=500,
    width=500,
)

fig.add_trace(
    go.Heatmap(
        x=x_plot,
        y=y_plot,
        z=analysis_functions.norm_data(d_plot),
        coloraxis="coloraxis",
    )
)

fig.update_coloraxes(cmin=0, cmax=1)

fig.show()

In [ ]:
## Stitch left and right side ##

x1, y1, dataslice1 = x_left, y_left, analysis_functions.norm_data(d_left)
x2, y2, dataslice2 = x_right, y_right, analysis_functions.norm_data(d_right) * 1.1

xs_3, ys_3, ds_3 = tr_functions.stitch_and_avg(
    x1,
    y1,
    dataslice1,
    x2,
    y2,
    dataslice2,
    no_avg=True,
)

In [ ]:
## Plot Data ##
x_plot, y_plot, d_plot = xs_3, ys_3, ds_3

fig = tr_functions.thesis_fig(
    title=f"E<sub>K</sub> = {slice_val}",
    xaxis_title=f"{Theta}",
    yaxis_title=f"{phi}",
    equiv_axes=False,
    height=500,
    width=500,
)

fig.add_trace(
    go.Heatmap(
        x=x_plot,
        y=y_plot,
        z=d_plot,
        coloraxis="coloraxis",
    )
)

fig.update_coloraxes(cmin=0, cmax=1)

fig.show()